## Data Preparation

 I am going to use data from Wikipedia for this analysis. Firstly,i am webscrapping data from wikipedia of "Special wards of Tokyo".This data has fields like Wards name, Pouplation of a particular ward, Major districts of the ward and its coordinates with area per square Killometer.  We will use  major populated districts to get the tragetted audience and area  per square Km to understand the land price in order to select the profitable venue.

Then we will use Foursquare location data in order to analyze the popular spots in major districts of Tokyo. In this i will analyze Top 5 major districts of Tokyo to know their poupular spots on the basis of highest foot traffic in those areas.  Depending on that we can choose perfect venue for the designer store.

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install folium
!pip install requests

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

In [2]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
import random
from IPython.display import Image 
from IPython.core.display import HTML 


from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup
from IPython.display import display_html
import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
source = requests.get('https://en.wikipedia.org/wiki/Special_wards_of_Tokyo').text
soup= BeautifulSoup(source,'lxml')
table_class="wikitable sortable jquery-tablesorter"
Wards_Tokyo_Table = soup.find('table', {'class':'wikitable sortable'})
print(Wards_Tokyo_Table)

<table class="wikitable sortable">
<tbody><tr>
<th>No.
</th>
<th class="unsortable">Flag
</th>
<th>Name
</th>
<th class="unsortable" width="55px">Kanji
</th>
<th>Population<br/>(as of October 2016<sup class="plainlinks noexcerpt noprint asof-tag update" style="display:none;"><a class="external text" href="https://en.wikipedia.org/w/index.php?title=Special_wards_of_Tokyo&amp;action=edit">[update]</a></sup>)
</th>
<th>Density<br/><span style="font-size:90%;">(/km<sup>2</sup>)</span>
</th>
<th>Area<br/><span style="font-size:90%;">(km<sup>2</sup>)</span>
</th>
<th class="unsortable">Major districts
</th></tr>
<tr>
<td>01</td>
<td><a class="image" href="/wiki/File:Flag_of_Chiyoda,_Tokyo.svg"><img alt="Flag of Chiyoda, Tokyo.svg" class="thumbborder" data-file-height="540" data-file-width="810" decoding="async" height="33" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Flag_of_Chiyoda%2C_Tokyo.svg/50px-Flag_of_Chiyoda%2C_Tokyo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commo

In [4]:
df=pd.read_html(str(Wards_Tokyo_Table))

df=pd.DataFrame(df[0])
print(df.head())

  No. Flag      Name Kanji  Population(as of October 2016  Density(/km2)  \
0  01  NaN   Chiyoda  千代田区                          59441           5100   
1  02  NaN      Chūō   中央区                         147620          14460   
2  03  NaN    Minato    港区                         248071          12180   
3  04  NaN  Shinjuku   新宿区                         339211          18620   
4  05  NaN    Bunkyō   文京区                         223389          19790   

   Area(km2)                                    Major districts  
0      11.66  Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,...  
1      10.21  Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb...  
2      20.37  Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong...  
3      18.22  Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich...  
4      11.29                              Hongō, Yayoi, Hakusan  


In [5]:
# drop the unwanted columns
data = df.drop(["No.", "Flag","Kanji","Density(/km2)"], inplace= True, axis=1)
df.head()

,Name,Population(as of October 2016,Area(km2),Major districts
0,Chiyoda,59441,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,..."
1,Chūō,147620,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb..."
2,Minato,248071,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong..."
3,Shinjuku,339211,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich..."
4,Bunkyō,223389,11.29,"Hongō, Yayoi, Hakusan"


In [8]:
new_name = {'Population(as of October 2016': 'Population', 'Name' : 'Ward', 'Major districts': 'Major_districts'}
df.rename(columns=new_name, inplace=True)
df.head()

,Ward,Population(as of October 2016,Area(km2),Major_districts
0,Chiyoda,59441,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,..."
1,Chūō,147620,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb..."
2,Minato,248071,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong..."
3,Shinjuku,339211,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich..."
4,Bunkyō,223389,11.29,"Hongō, Yayoi, Hakusan"


In [9]:
df.head(23)

,Ward,Population(as of October 2016,Area(km2),Major_districts
0,Chiyoda,59441,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,..."
1,Chūō,147620,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb..."
2,Minato,248071,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong..."
3,Shinjuku,339211,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich..."
4,Bunkyō,223389,11.29,"Hongō, Yayoi, Hakusan"
5,Taitō,200486,10.11,"Ueno, Asakusa"
6,Sumida,260358,13.77,"Kinshichō, Morishita, Ryōgoku"
7,Kōtō,502579,40.16,"Kiba, Ariake, Kameido, Tōyōchō, Monzennakachō,..."
8,Shinagawa,392492,22.84,"Shinagawa, Gotanda, Ōsaki, Hatanodai, Ōimachi,..."
9,Meguro,280283,14.67,"Meguro, Nakameguro, Jiyugaoka, Komaba, Aobadai"


In [12]:
df.shape

(24, 5)

In [13]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Tokyo_explorer")
df['Major_districts_coord']=df['Ward'].apply(geolocator.geocode)
df.head(23)

,Ward,Population(as of October 2016,Area(km2),Major_districts,Major_districts_coord
0,Chiyoda,59441,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,...","(千代田区, 東京都, 日本, (35.6938097, 139.7532163))"
1,Chūō,147620,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb...","(中央区, 東京都, 日本, (35.666255, 139.775565))"
2,Minato,248071,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong...","(港区, 東京都, 日本, (35.6432274, 139.7400553))"
3,Shinjuku,339211,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich...","(新宿区, 東京都, 日本, (35.6937632, 139.7036319))"
4,Bunkyō,223389,11.29,"Hongō, Yayoi, Hakusan","(文京区, 東京都, 日本, (35.71881, 139.744732))"
5,Taitō,200486,10.11,"Ueno, Asakusa","(台東区, 東京都, 日本, (35.71745, 139.790859))"
6,Sumida,260358,13.77,"Kinshichō, Morishita, Ryōgoku","(墨田区, 東京都, 日本, (35.700429, 139.805017))"
7,Kōtō,502579,40.16,"Kiba, Ariake, Kameido, Tōyōchō, Monzennakachō,...","(江東区, 東京都, 日本, (35.649154, 139.81279))"
8,Shinagawa,392492,22.84,"Shinagawa, Gotanda, Ōsaki, Hatanodai, Ōimachi,...","(品川区, 東京都, 日本, (35.599252, 139.73891))"
9,Meguro,280283,14.67,"Meguro, Nakameguro, Jiyugaoka, Komaba, Aobadai","(目黒区, 東京都, 日本, (35.62125, 139.688014))"


In [12]:
df['Major_districts_coord'] = df['Major_districts_coord'].apply(lambda x: (x.latitude, x.longitude))
df.head(23)

,Ward,Population(as of October 2016,Area(km2),Major_districts,Major_districts_coord
0,Chiyoda,59441,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,...","(35.6938097, 139.7532163)"
1,Chūō,147620,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb...","(35.666255, 139.775565)"
2,Minato,248071,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong...","(35.6432274, 139.7400553)"
3,Shinjuku,339211,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich...","(35.6937632, 139.7036319)"
4,Bunkyō,223389,11.29,"Hongō, Yayoi, Hakusan","(35.71881, 139.744732)"
5,Taitō,200486,10.11,"Ueno, Asakusa","(35.71745, 139.790859)"
6,Sumida,260358,13.77,"Kinshichō, Morishita, Ryōgoku","(35.700429, 139.805017)"
7,Kōtō,502579,40.16,"Kiba, Ariake, Kameido, Tōyōchō, Monzennakachō,...","(35.649154, 139.81279)"
8,Shinagawa,392492,22.84,"Shinagawa, Gotanda, Ōsaki, Hatanodai, Ōimachi,...","(35.599252, 139.73891)"
9,Meguro,280283,14.67,"Meguro, Nakameguro, Jiyugaoka, Komaba, Aobadai","(35.62125, 139.688014)"
